# Example: Encoders / Batch Encoder

In `1_bfv_basics.ipynb` we showed how to perform a very simple computation using the
BFV scheme. The computation was performed modulo the plain_modulus parameter, and
utilized only one coefficient from a BFV plaintext polynomial. This approach has
two notable problems:

  1. Practical applications typically use integer or real number arithmetic,
    not modular arithmetic.
  2. We used only one coefficient of the plaintext polynomial. This is really
    wasteful, as the plaintext polynomial is large and will in any case be
    encrypted in its entirety.

For (1), one may ask why not just increase the plain_modulus parameter until no
overflow occurs, and the computations behave as in integer arithmetic. The problem
is that increasing plain_modulus increases noise budget consumption, and decreases
the initial noise budget too.

In these examples we will discuss other ways of laying out data into plaintext
elements (encoding) that allow more computations without data type overflow, and
can allow the full plaintext polynomial to be utilized.

In [1]:
import seal
import example_helper

Let `N` denote the `poly_modulus_degree` and `T` denote the `plain_modulus`. Batching
allows the BFV plaintext polynomials to be viewed as `2-by-(N/2)` matrices, with
each element an integer modulo `T`. In the matrix view, encrypted operations act
element-wise on encrypted matrices, allowing the user to obtain speeds-ups of
several orders of magnitude in fully vectorizable computations. Thus, in all
but the simplest computations, batching should be the preferred method to use
with BFV, and when used properly will result in implementations outperforming
anything done without batching.

In a later example, we will demonstrate how to use the BGV scheme. Batching
works similarly for the BGV scheme to this example for the BFV scheme. For example,
simply changing `scheme_type.bfv` into `scheme_type.bgv` can make this example
work for the BGV scheme.

In [2]:
parms = seal.EncryptionParameters(seal.scheme_type.bfv)
poly_modulus_degree = 8192
parms.poly_modulus_degree = poly_modulus_degree
parms.coeff_modulus = seal.CoeffModulus.BFVDefault(poly_modulus_degree)

To enable batching, we need to set the `plain_modulus` to be a prime number
congruent to `1 modulo 2*poly_modulus_degree`. Microsoft SEAL provides a helper
method for finding such a prime. In this example we create a 20-bit prime
that supports batching.

In [3]:
parms.plain_modulus = seal.PlainModulus.Batching(poly_modulus_degree, 20)

context = seal.SEALContext(parms)
example_helper.print_parameters(context)

/
| Encryption parameters
| scheme: bfv
| poly_modulus_degree: 8192
| coeff_modulus size: 218(43 + 43 + 44 + 44 + 44) bits
| plain_modulus: 1032193
\


We can verify that batching is indeed enabled by looking at the encryption
parameter qualifiers created by SEALContext.

In [4]:
qualifiers = context.first_context_data.qualifiers
print(f"Batching enabled: {qualifiers.using_batching}")

Batching enabled: True


In [5]:
keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()
encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

Batching is done through an instance of the BatchEncoder class.

In [6]:
batch_encoder = seal.BatchEncoder(context)

The total number of batching `slots` equals the `poly_modulus_degree`, `N`, and
these slots are organized into `2-by-(N/2)` matrices that can be encrypted and
computed on. Each slot contains an integer modulo `plain_modulus`.

In [7]:
slot_count = batch_encoder.slot_count
row_size = slot_count // 2
print(f"Plaintext matrix row size: {row_size}")

Plaintext matrix row size: 4096


The matrix plaintext is simply given to BatchEncoder as a flattened vector
of numbers. The first `row_size` many numbers form the first row, and the
rest form the second row. Here we create the following matrix:

    [ 0,  1,  2,  3,  0,  0, ...,  0 ]
    [ 4,  5,  6,  7,  0,  0, ...,  0 ]

In [8]:
pod_matrix = [0 for _ in range(slot_count)]
pod_matrix[0] = 0
pod_matrix[1] = 1
pod_matrix[2] = 2
pod_matrix[3] = 3
pod_matrix[row_size] = 4
pod_matrix[row_size+1] = 5
pod_matrix[row_size+2] = 6
pod_matrix[row_size+3] = 7

print("Input plaintext matrix:")
example_helper.print_matrix(pod_matrix, row_size)


Input plaintext matrix:

    [ 0.000, 1.000, 2.000, 3.000, 0.000, ..., 0.000, 0.000, 0.000, 0.000, 0.000 ]
    [ 4.000, 5.000, 6.000, 7.000, 0.000, ..., 0.000, 0.000, 0.000, 0.000, 0.000 ]



First we use BatchEncoder to encode the matrix into a plaintext polynomial.

In [9]:
print("Encode plaintext matrix.")
plain_matrix = batch_encoder.encode(pod_matrix)

Encode plaintext matrix.


We can instantly decode to verify correctness of the encoding. Note that no
encryption or decryption has yet taken place.

In [10]:
pod_result = batch_encoder.decode(plain_matrix)
example_helper.print_matrix(pod_result, row_size)


    [ 0.000, 1.000, 2.000, 3.000, 0.000, ..., 0.000, 0.000, 0.000, 0.000, 0.000 ]
    [ 4.000, 5.000, 6.000, 7.000, 0.000, ..., 0.000, 0.000, 0.000, 0.000, 0.000 ]



In [11]:
print("Encrypt plain_matrix to encrypted_matrix.")
encrypted_matrix = encryptor.encrypt(plain_matrix)
print(f"noise budget in encrypted_matrix: {decryptor.invariant_noise_budget(encrypted_matrix)} bits")

Encrypt plain_matrix to encrypted_matrix.
noise budget in encrypted_matrix: 146 bits


Operating on the ciphertext results in homomorphic operations being performed
simultaneously in all 8192 slots (matrix elements). To illustrate this, we
form another plaintext matrix

    [ 1,  2,  1,  2,  1,  2, ..., 2 ]
    [ 1,  2,  1,  2,  1,  2, ..., 2 ]
    
and encode it into a plaintext.

In [12]:
pod_matrix2 = [(i & 1) + 1 for i in range(slot_count)]
plain_matrix2 = batch_encoder.encode(pod_matrix2)

print("Second input plaintext matirx:")
example_helper.print_matrix(pod_matrix2, row_size)

Second input plaintext matirx:

    [ 1.000, 2.000, 1.000, 2.000, 1.000, ..., 2.000, 1.000, 2.000, 1.000, 2.000 ]
    [ 1.000, 2.000, 1.000, 2.000, 1.000, ..., 2.000, 1.000, 2.000, 1.000, 2.000 ]



In [13]:
print("Sum, square, and relinearize.")
evaluator.add_plain_inplace(encrypted_matrix, plain_matrix2)
evaluator.square_inplace(encrypted_matrix)
evaluator.relinearize_inplace(encrypted_matrix, relin_keys)

print(f"noise budget in result: {decryptor.invariant_noise_budget(encrypted_matrix)} bits")

Sum, square, and relinearize.
noise budget in result: 114 bits


We decrypt and decompose the plaintext to recover the result as a matrix.

In [14]:
print("Decrypt and decode result.")
plain_result = decryptor.decrypt(encrypted_matrix)
pod_result = batch_encoder.decode(plain_result)
example_helper.print_matrix(pod_result, row_size)

Decrypt and decode result.

    [ 1.000, 9.000, 9.000, 25.000, 1.000, ..., 4.000, 1.000, 4.000, 1.000, 4.000 ]
    [ 25.000, 49.000, 49.000, 81.000, 1.000, ..., 4.000, 1.000, 4.000, 1.000, 4.000 ]



Batching allows us to efficiently use the full plaintext polynomial when the
desired encrypted computation is highly parallelizable. However, it has not
solved the other problem mentioned in the beginning of this file: each slot
holds only an integer modulo `plain_modulus`, and unless `plain_modulus` is very
large, we can quickly encounter data type overflow and get unexpected results
when integer computations are desired. Note that overflow cannot be detected
in encrypted form. The CKKS scheme (and the CKKSEncoder) addresses the data
type overflow issue, but at the cost of yielding only approximate results.